In [2]:
import boto3
import pandas as pd
from io import StringIO , BytesIO
import os, glob
from datetime import datetime, timedelta

In [3]:
arg_date = '2022-12-30'

In [4]:
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

In [5]:
arg_date_dt

datetime.date(2022, 12, 29)

In [6]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('xetra-1234')
# buck_all_obj = bucket.objects.all()


In [7]:
# all_objects = list(bucket.objects.all())
# last_object_by_key = sorted(all_objects, key=lambda obj: obj.key)[-1]

# print("Last object by key:", last_object_by_key.key)

In [8]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() >= arg_date_dt]

In [9]:
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-29/2022-12-29_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-29/2022-12-29_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-29/2022-12-29_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-29/2022-12-29_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-29/2022-12-29_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-29/2022-12-29_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-29/2022-12-29_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-29/2022-12-29_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-29/2022-12-29_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-29/2022-12-29_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-29/2022-12-29_BINS_XETR10.csv'),
 s3.Object

In [10]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')

In [11]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

Less efficient method (not pythonic) -> more efficient pd.concat([pd.DataFrame([i], columns=['A']) for i in range(5)],
          ignore_index=True)

In [12]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data,delimiter=',')
    if not df.empty and not df.isna().all().all():
        df_all = pd.concat([df_all, df], ignore_index=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_2264\1393382227.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all, df], ignore_index=True)


In [13]:
df_all.sort_values(by=['ISIN','Date'])

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
3135,AT000000STR1,XD4,STRABAG SE,Common stock,EUR,7026002,2022-12-29,08:06,36.6000,36.6000,36.6000,36.6000,118,1
12433,AT000000STR1,XD4,STRABAG SE,Common stock,EUR,7026002,2022-12-29,08:37,36.3000,36.3000,36.3000,36.3000,107,1
17595,AT000000STR1,XD4,STRABAG SE,Common stock,EUR,7026002,2022-12-29,08:56,36.1000,36.1000,36.1000,36.1000,60,1
20877,AT000000STR1,XD4,STRABAG SE,Common stock,EUR,7026002,2022-12-29,09:09,36.0000,36.0000,36.0000,36.0000,100,2
21425,AT000000STR1,XD4,STRABAG SE,Common stock,EUR,7026002,2022-12-29,09:11,35.7500,35.7500,35.7500,35.7500,80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194860,XS2434891219,FBTC,FIDELITY EXC OPEN END ETP,ETN,EUR,7383398,2022-12-30,12:12,3.4184,3.4184,3.4184,3.4184,0,1
263708,XS2434891219,FBTC,FIDELITY EXC OPEN END ETP,ETN,EUR,7383398,2022-12-30,16:36,3.6620,3.6620,3.6620,3.6620,0,2
266323,XS2434891219,FBTC,FIDELITY EXC OPEN END ETP,ETN,EUR,7383398,2022-12-31,08:04,3.4412,3.4412,3.4412,3.4412,0,1
326779,XS2434891219,FBTC,FIDELITY EXC OPEN END ETP,ETN,EUR,7383398,2022-12-31,12:12,3.4184,3.4184,3.4184,3.4184,0,1


In [14]:
columns = ["ISIN", "Mnemonic", "Date", "Time", "StartPrice", "EndPrice", "MinPrice", "MaxPrice", "TradedVolume"]
df_all = df_all.loc[:, columns]

In [15]:
df_all

,ISIN,Mnemonic,Date,Time,StartPrice,EndPrice,MinPrice,MaxPrice,TradedVolume
0,AT0000A0E9W5,SANT,2022-12-29,08:00,13.880,13.880,13.880,13.880,63
1,DE000A0DJ6J9,S92,2022-12-29,08:00,29.000,29.140,29.000,29.140,6004
2,DE000A0D6554,NDX1,2022-12-29,08:00,13.760,13.800,13.760,13.880,49560
3,DE000A0D9PT0,MTX,2022-12-29,08:00,212.000,210.600,210.400,212.000,17455
4,DE000A0JL9W6,VBK,2022-12-29,08:00,66.200,66.250,65.350,66.250,10884
...,...,...,...,...,...,...,...,...,...
395752,GB00BLD4ZP54,CLTC,2022-12-31,16:46,19.324,19.324,19.324,19.324,0
395753,LU1923627332,RUSL,2022-12-31,16:52,12.400,12.400,12.400,12.400,2645
395754,US98956P1021,ZIM,2022-12-31,20:30,113.100,113.100,113.100,113.100,0
395755,US9224171002,VEO,2022-12-31,20:30,24.600,24.600,24.600,24.600,0


In [16]:
df_all.dropna(inplace=True)

In [17]:
df_all.shape

(395757, 9)

## Get opening price per ISIN and day

In [18]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [19]:
df_all[df_all['ISIN']=='DE0007568578']

,ISIN,Mnemonic,Date,Time,StartPrice,EndPrice,MinPrice,MaxPrice,TradedVolume,opening_price
200,DE0007568578,F3C,2022-12-29,08:02,21.65,22.25,21.65,22.25,4590,21.65
373,DE0007568578,F3C,2022-12-29,08:03,22.10,22.05,22.05,22.30,2474,21.65
663,DE0007568578,F3C,2022-12-29,08:04,22.15,22.05,22.05,22.15,764,21.65
2941,DE0007568578,F3C,2022-12-29,08:06,22.15,22.05,22.05,22.20,1182,21.65
3266,DE0007568578,F3C,2022-12-29,08:07,22.20,22.20,22.20,22.20,500,21.65
...,...,...,...,...,...,...,...,...,...,...
390392,DE0007568578,F3C,2022-12-31,16:24,25.05,25.05,25.05,25.05,8,21.65
391754,DE0007568578,F3C,2022-12-31,16:28,24.90,25.05,24.90,25.05,597,21.65
392134,DE0007568578,F3C,2022-12-31,16:29,25.05,24.95,24.95,25.05,392,21.65
392433,DE0007568578,F3C,2022-12-31,16:30,24.80,24.80,24.75,24.80,134,21.65


## Get closing price per ISIN and day

In [20]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [21]:
df_all[df_all['ISIN']=='DE0007568578']

,ISIN,Mnemonic,Date,Time,StartPrice,EndPrice,MinPrice,MaxPrice,TradedVolume,opening_price,closing_price
200,DE0007568578,F3C,2022-12-29,08:02,21.65,22.25,21.65,22.25,4590,21.65,25.25
373,DE0007568578,F3C,2022-12-29,08:03,22.10,22.05,22.05,22.30,2474,21.65,25.25
663,DE0007568578,F3C,2022-12-29,08:04,22.15,22.05,22.05,22.15,764,21.65,25.25
2941,DE0007568578,F3C,2022-12-29,08:06,22.15,22.05,22.05,22.20,1182,21.65,25.25
3266,DE0007568578,F3C,2022-12-29,08:07,22.20,22.20,22.20,22.20,500,21.65,25.25
...,...,...,...,...,...,...,...,...,...,...,...
390392,DE0007568578,F3C,2022-12-31,16:24,25.05,25.05,25.05,25.05,8,21.65,25.25
391754,DE0007568578,F3C,2022-12-31,16:28,24.90,25.05,24.90,25.05,597,21.65,25.25
392134,DE0007568578,F3C,2022-12-31,16:29,25.05,24.95,24.95,25.05,392,21.65,25.25
392433,DE0007568578,F3C,2022-12-31,16:30,24.80,24.80,24.75,24.80,134,21.65,25.25


## Aggregations

In [22]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [23]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT000000STR1,2022-12-29,36.6000,36.700,35.7500,36.700,1773
1,AT000000STR1,2022-12-30,36.6000,36.700,35.7500,36.700,1773
2,AT000000STR1,2022-12-31,36.6000,36.700,35.7500,36.700,1773
3,AT00000FACC2,2022-12-29,8.0500,8.570,7.8700,8.570,10205
4,AT00000FACC2,2022-12-30,8.0500,8.570,7.8700,8.570,10205
...,...,...,...,...,...,...,...
9691,XS2376095068,2022-12-30,34.2880,36.500,34.0640,36.500,1000
9692,XS2376095068,2022-12-31,34.2880,36.500,34.0640,36.500,1000
9693,XS2434891219,2022-12-29,3.4412,3.662,3.4184,3.662,0
9694,XS2434891219,2022-12-30,3.4412,3.662,3.4184,3.662,0


percent change prev closing

In [24]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [25]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,prev_closing_price
0,AT000000STR1,2022-12-29,36.6000,36.700,35.7500,36.700,1773,NaN
1,AT000000STR1,2022-12-30,36.6000,36.700,35.7500,36.700,1773,36.700
2,AT000000STR1,2022-12-31,36.6000,36.700,35.7500,36.700,1773,36.700
3,AT00000FACC2,2022-12-29,8.0500,8.570,7.8700,8.570,10205,NaN
4,AT00000FACC2,2022-12-30,8.0500,8.570,7.8700,8.570,10205,8.570
...,...,...,...,...,...,...,...,...
9691,XS2376095068,2022-12-30,34.2880,36.500,34.0640,36.500,1000,36.500
9692,XS2376095068,2022-12-31,34.2880,36.500,34.0640,36.500,1000,36.500
9693,XS2434891219,2022-12-29,3.4412,3.662,3.4184,3.662,0,NaN
9694,XS2434891219,2022-12-30,3.4412,3.662,3.4184,3.662,0,3.662


In [26]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [27]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [28]:
df_all = df_all.round(decimals=2)

In [29]:
df_all.reset_index(inplace=True)

In [33]:
df_all = df_all[df_all.Date >= arg_date]

In [34]:
df_all

,index,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
1,1,AT000000STR1,2022-12-30,36.60,36.70,35.75,36.70,1773,0.0
2,2,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,0.0
4,4,AT00000FACC2,2022-12-30,8.05,8.57,7.87,8.57,10205,0.0
5,5,AT00000FACC2,2022-12-31,8.05,8.57,7.87,8.57,10205,0.0
7,7,AT0000606306,2022-12-30,14.51,15.00,13.65,15.28,107836,0.0
...,...,...,...,...,...,...,...,...,...
9689,9689,XS2314660700,2022-12-31,22.26,21.92,21.92,22.28,0,0.0
9691,9691,XS2376095068,2022-12-30,34.29,36.50,34.06,36.50,1000,0.0
9692,9692,XS2376095068,2022-12-31,34.29,36.50,34.06,36.50,1000,0.0
9694,9694,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.0


##lecture 13

## Write to S3

In [36]:
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [37]:
key

'xetra_daily_report_20241005_051801.parquet'

In [38]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket('ayush-5-oct-24')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='ayush-5-oct-24', key='xetra_daily_report_20241005_051801.parquet')

##read the uploaded file

In [40]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20241005_051801.parquet


In [41]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20241005_051801.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [42]:
df_report

,index,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,1,AT000000STR1,2022-12-30,36.60,36.70,35.75,36.70,1773,0.0
1,2,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,0.0
2,4,AT00000FACC2,2022-12-30,8.05,8.57,7.87,8.57,10205,0.0
3,5,AT00000FACC2,2022-12-31,8.05,8.57,7.87,8.57,10205,0.0
4,7,AT0000606306,2022-12-30,14.51,15.00,13.65,15.28,107836,0.0
...,...,...,...,...,...,...,...,...,...
6459,9689,XS2314660700,2022-12-31,22.26,21.92,21.92,22.28,0,0.0
6460,9691,XS2376095068,2022-12-30,34.29,36.50,34.06,36.50,1000,0.0
6461,9692,XS2376095068,2022-12-31,34.29,36.50,34.06,36.50,1000,0.0
6462,9694,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.0
